In [1]:
# Training variables
rolling_mean = 7
lag = 8
nobs = 30

In [2]:
import sys
sys.path.insert(0, '/home/khiem/Dropbox/Education/Drexel/DSCI-591/Project/Walmart-Strategic-Sales-Forecasting')

In [3]:
def invert_transformation(df_train, df_forecast):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [29]:
import boto3
import pandas as pd
from itertools import product
from data_processing.pre_modeling import pre_modeling
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from pathlib import Path

DATA_FOLDER = Path('/home/khiem/Dropbox/Education/Drexel/DSCI-591/Project/Walmart-Strategic-Sales-Forecasting/data/processed/')
STATES = ['CA', 'WI', 'TX']
STORES = [1, 2, 3, 4]

bucket = 'dsci-632'

scores = []
feature_correlations = pd.DataFrame()
for store in product(STATES, STORES):
    try:
        timeseries_cols = ['cpi', 'unemployment_rate', 'gas_price', store[0].lower()+'_walmart', store[0].lower()+'_hobbies', store[0].lower()+'_household', store[0].lower()+'_foods', 'median_price', 'min_price', 'max_price']
        time_cols = ['wm_yr_wk', 'wday', 'day', 'month', 'year']
        target_col = ['store_sales']
        store_id = f'{store[0].upper()}_{store[1]}'
        store_data = DATA_FOLDER / Path(f'{store_id}_sales_data.csv')
        df = pd.read_csv(store_data, index_col=0, parse_dates=True)
        df_processed = pre_modeling(df)
        df_processed = df_processed.rolling(rolling_mean).mean().dropna()
        df_used = df_processed[timeseries_cols+target_col]
        df_original = df_used
        df_test = df_used[-nobs:]
        df_train = df_used[0:-nobs]
        df_train = df_train.diff().dropna()
        model = VAR(df_train)
        model_fitted = model.fit(lag)
        lag_order = model_fitted.k_ar
        forecast_input = df_train.values[-lag_order:]
        fc = model_fitted.forecast(y=forecast_input, steps=nobs)
        df_forecast = pd.DataFrame(fc, index=df_test.index, columns=df_test.columns + '_1d')
        df_results = invert_transformation(df_original[0:-nobs], df_forecast)
        # export to file
        df_results.to_csv(DATA_FOLDER / Path(f'{store_id}_sales_forecast.csv'))
        
        plt.figure(figsize=(9, 3))
        plt.plot(df_results.index, df_results["store_sales_forecast"], label = "forecast")
        plt.plot(df_test.index, df_test["store_sales"], label = "actual")
        plt.legend()
        plt.show()
        
        plt.figure(figsize=(9, 3))
        plt.plot(df_results.index, df_results["store_sales_forecast"], label = "forecast")
        plt.plot(df_original[0:-nobs].index, df_original[0:-nobs]["store_sales"], label = "past data")
        plt.plot(df_test.index, df_test["store_sales"], label = "actual")
        plt.legend()
        plt.show()
    except FileNotFoundError:
        pass


/home/khiem/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


TypeError: Index(...) must be called with a collection of some kind, 'CA' was passed